In [1]:
# import bcolz 
import importlib
import numpy as np
import torch.utils.data
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

from torch.autograd import Variable
import torch# import bcolz 
import importlib
import numpy as np
import torch.utils.data
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from torch.autograd import Variable
import torch

In [2]:
import torch.nn as nn
class SEWeightModule(nn.Module):
    def __init__(self, channels, reduction=4):
        super(SEWeightModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc1 = nn.Linear(channels,channels//reduction)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(channels//reduction, channels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.avg_pool(x)
        a,b,c, = x.size()
        out = out.view(a, b)
        
        
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        weight = self.sigmoid(out).view(a, b,1)
        return weight

In [3]:
class identity_block(nn.Module):
    def __init__(self,channels_in,channels_out,ra):
        super(identity_block,self).__init__()
        self.channels_in = channels_in
        self.channels_out = channels_out
        self.stride = 1
        self.kernel_size = 3
        self.ra=ra
        self.conv1 = nn.Conv1d(in_channels=self.channels_in,out_channels=self.channels_out,kernel_size=1,stride=1, padding=0)
        self.conv2 = nn.Conv1d(in_channels=self.channels_out,out_channels=self.channels_out,kernel_size=3,stride=1, padding=1)
        self.prl = nn.ReLU()
        self.conv11 = nn.Conv1d(in_channels=self.channels_out,out_channels=self.channels_out//4,kernel_size=self.kernel_size,stride=self.stride, padding=1,dilation=1)
        self.conv12 = nn.Conv1d(in_channels=self.channels_out,out_channels=self.channels_out//4,kernel_size=self.kernel_size,stride=self.stride, padding=3,dilation=3)
        self.conv13 = nn.Conv1d(in_channels=self.channels_out,out_channels=self.channels_out//4,kernel_size=self.kernel_size,stride=self.stride, padding=5,dilation=5)
        self.conv14 = nn.Conv1d(in_channels=self.channels_out,out_channels=self.channels_out//4,kernel_size=self.kernel_size,stride=self.stride, padding=7,dilation=7)
        self.se = SEWeightModule(self.channels_out,self.ra)

        self.conv3 = nn.Conv1d(in_channels=self.channels_out,out_channels=self.channels_out,kernel_size=15,stride=1, padding=7)
        self.rl = nn.ReLU()
        self.maxp = nn.MaxPool1d(kernel_size=2, stride=2)

    
    def forward(self, x):
        x=self.conv1(x)
        x1=x
        x=self.prl(self.conv2(x))
        
        x11=self.conv11(x)
        x12=self.conv12(x)
        x13=self.conv13(x)
        x14=self.conv14(x)
        
        output = torch.cat([x11,x12,x13,x14],1)
        Woutput= self.se(output )

        Woutput=Woutput* output +x1

        Woutput=self.rl(Woutput)
        Woutput=self.maxp(Woutput)
        
        return Woutput

class identity_block3(nn.Module):
    def __init__(self,channels_in,channels_out,ra):
        super(identity_block3,self).__init__()
        self.channels_in = channels_in
        self.channels_out = channels_out
        self.stride = 1
        self.kernel_size = 3
        self.ra=ra
        
        self.conv1 = nn.ConvTranspose1d(in_channels=self.channels_in,out_channels=self.channels_out,kernel_size=4,stride=2, padding=1)
        
        self.conv2 = nn.ConvTranspose1d(in_channels=self.channels_out,out_channels=self.channels_out,kernel_size=3,stride=1, padding=1)
        self.prl = nn.ReLU()
        
        self.conv11 = nn.ConvTranspose1d(in_channels=self.channels_out,out_channels=self.channels_out//4,kernel_size=self.kernel_size,stride=self.stride, padding=1,dilation=1)
        self.conv12 = nn.ConvTranspose1d(in_channels=self.channels_out,out_channels=self.channels_out//4,kernel_size=self.kernel_size,stride=self.stride, padding=3,dilation=3)
        self.conv13 = nn.ConvTranspose1d(in_channels=self.channels_out,out_channels=self.channels_out//4,kernel_size=self.kernel_size,stride=self.stride, padding=5,dilation=5)
        self.conv14 = nn.ConvTranspose1d(in_channels=self.channels_out,out_channels=self.channels_out//4,kernel_size=self.kernel_size,stride=self.stride, padding=7,dilation=7)
                           
    
        self.se = SEWeightModule(self.channels_out,self.ra)

        
        self.conv3 = nn.Conv1d(in_channels=self.channels_out,out_channels=self.channels_out,kernel_size=1,stride=1, padding=0)
        self.pr2 = nn.ReLU()


    
    def forward(self, x):
        x=self.conv1(x)
        x1=x
        x=self.prl(self.conv2(x))
        
        x11=self.conv11(x)
        x12=self.conv12(x)
        x13=self.conv13(x)
        x14=self.conv14(x)
        
        output = torch.cat([x11,x12,x13,x14],1)
        Woutput= self.se(output )

        Woutput=Woutput* output +x1
  
        
        Woutput=self.pr2(Woutput)
        Woutput=self.conv3(Woutput)
  
        
        return Woutput

class identity_block4(nn.Module):
    def __init__(self,channels_in,channels_out,ra):
        super(identity_block4,self).__init__()
        self.channels_in = channels_in
        self.channels_out = channels_out
        self.stride = 1
        self.kernel_size = 3
        self.ra=ra
        
        self.conv1 = nn.ConvTranspose1d(in_channels=self.channels_in,out_channels=self.channels_out,kernel_size=4,stride=2, padding=1)
        
        self.conv2 = nn.ConvTranspose1d(in_channels=self.channels_out,out_channels=self.channels_out,kernel_size=3,stride=1, padding=1)
        self.prl = nn.ReLU()
        
        self.conv11 = nn.ConvTranspose1d(in_channels=self.channels_out,out_channels=self.channels_out,kernel_size=self.kernel_size,stride=self.stride, padding=1,dilation=1)

        
        self.conv3 = nn.Conv1d(in_channels=self.channels_out,out_channels=self.channels_out,kernel_size=1,stride=1, padding=0)
        self.pr2 = nn.ReLU()


    
    def forward(self, x):
        x=self.conv1(x)
        x1=x
        x=self.prl(self.conv2(x))
        
        x11=self.conv11(x)+x1


        Woutput=self.pr2(x11)
        Woutput=self.conv3(Woutput)
  
        
        return Woutput



In [4]:
S_in=8
class DAE(nn.Module):
    def __init__(self):
        super(DAE,self).__init__()
        self.A1 = identity_block(1,16,2)
        self.A2 = identity_block(16,32,4)
        self.A3 = identity_block(32,64,4)
        self.A4 = identity_block(64,128,8)
        
        self.B1 = identity_block3(128,64,4)
        self.B2 = identity_block3(64,32,4)
        self.B3 = identity_block3(32,16,2)
        self.B4 = identity_block4(16,8,2)
        
        self.B5= nn.ConvTranspose1d(8,1,3,1,1)
        self.SG = nn.Sigmoid()
        self.init_weights()
        
        self.lstm = nn.LSTM(input_size=S_in, hidden_size=S_in, num_layers=1, batch_first=True,bidirectional=True)
        self.fc1 = nn.Linear(S_in*8,S_in)
   
    def init_weights(self):
        """
        Initialize weights for convolution layers using Xavier initialization.
        """
        for m in self.modules():
            if isinstance(m, nn.Conv1d) or isinstance(m, nn.ConvTranspose1d):
                nn.init.xavier_normal(m.weight.data)      
    
    def forward(self, x):
       
        x1=self.A1(x)
        x2=self.A2(x1)
        x3=self.A3(x2)
        x4=self.A4(x3)
        h1,h2,h3=x4.shape
        x44=x4.reshape(h1*h2,16,8)
        x44, h=self.lstm(x44)
        o1,o2=torch.split(x44,S_in,2)
        x44=(o1+o2)/2
        x44=x44.reshape(h1,h2,h3)
        x5=self.B1(x44)
        x5=x5+x3
        x6=self.B2(x5)
        x6=x6+x2
        x7=self.B3(x6)
        x6=x7+x1
        x8=self.B4(x7)
        x9=self.B5(x8)        
        return self.SG(x9)

In [5]:
model = DAE()
from thop import profile
model = model.to('cpu')
input = torch.randn(1,1,2048)
flops, params = profile(model, inputs=(input, ))
print(params,flops)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool1d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.activation.Sigmoid'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class '__main__.SEWeightModule'>. Treat it as zero Macs and zero Params.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool1d'>.
[WARN] Cannot find rule for <class '__main__.identity_block'>. Treat it as zero Macs and zero Params.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.ConvTranspose1d'>.
[WARN] Cannot find rule for <class '__main__.identity_block3'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class '__main__.identity_block4'>. Treat it as zero Macs and zero P

E:\Anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:28: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
E:\Anaconda3\envs\torch\lib\site-packages\thop\vision\basic_hooks.py:92: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  kernel = torch.DoubleTensor([*(x[0].shape[2:])]) // torch.DoubleTensor(list((m.output_size,))).squeeze()


In [6]:
ONEDAMRAE = DAE()
ONEDAMRAE.load_state_dict(torch.load('C:/Users/cccc/Desktop/1-DAMRAE/WEIGHT.pth'))
ONEDAMRAE.cpu().eval

E:\Anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:28: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.


<bound method Module.eval of DAE(
  (A1): identity_block(
    (conv1): Conv1d(1, 16, kernel_size=(1,), stride=(1,))
    (conv2): Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=(1,))
    (prl): ReLU()
    (conv11): Conv1d(16, 4, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv12): Conv1d(16, 4, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
    (conv13): Conv1d(16, 4, kernel_size=(3,), stride=(1,), padding=(5,), dilation=(5,))
    (conv14): Conv1d(16, 4, kernel_size=(3,), stride=(1,), padding=(7,), dilation=(7,))
    (se): SEWeightModule(
      (avg_pool): AdaptiveAvgPool1d(output_size=1)
      (fc1): Linear(in_features=16, out_features=8, bias=True)
      (relu): ReLU()
      (fc2): Linear(in_features=8, out_features=16, bias=True)
      (sigmoid): Sigmoid()
    )
    (conv3): Conv1d(16, 16, kernel_size=(15,), stride=(1,), padding=(7,))
    (rl): ReLU()
    (maxp): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (A2): identity_

In [7]:
from scipy.io import loadmat
import numpy as np

dict_mat = loadmat("C:\\Users\\cccc\\Desktop\\1-DAMRAE\\TEST_NAM_SJR_neg10_SNR10.mat")
print(type(dict_mat))
print(dict_mat.keys())
test = np.matrix.transpose(dict_mat["test"])    #echo+AWGN+jamming
dict_mat = loadmat("C:\\Users\\cccc\\Desktop\\1-DAMRAE\\pure.mat")
testb = np.matrix.transpose(dict_mat["testb"])  #pure echo

S1=test*2-1  #Normalized → -1 to 1
S2=testb*2-1
P1=np.zeros(len(S1))
for i in range(len(S1)):
    P1[i]=np.abs(np.mean(S1[i,:]*S2[i,:]))/np.sqrt(np.abs(np.mean(S1[i,:]*S1[i,:]))*np.abs(np.mean(S2[i,:]*S2[i,:])))
print(np.mean(P1)) #   Waveform similarity between the received signal and the pure echo

test=torch.from_numpy(test).float().view(len(S1),1,2048)
decoder_output= ONEDAMRAE(test)
decoder_output = decoder_output.detach().cpu().numpy()
decoder_output.shape
from scipy.io import savemat
decoder_output.shape
import numpy as np
from scipy import io

S3=decoder_output*2-1
P2=np.zeros(len(S2))
for i in range(len(S2)):
    P2[i]=np.abs(np.mean(S2[i,:]*S3[i,:]))/np.sqrt(np.abs(np.mean(S2[i,:]*S2[i,:]))*np.abs(np.mean(S3[i,:]*S3[i,:])))
print(np.mean(P2)) #   Waveform similarity between the recovered signal and the pure echo

<class 'dict'>
dict_keys(['__header__', '__version__', '__globals__', 'test'])
0.2756278725995399
0.9843764258662282


In [8]:
from scipy.io import loadmat
import numpy as np

dict_mat = loadmat("C:\\Users\\cccc\\Desktop\\1-DAMRAE\\TEST_NFM_SJR_neg10_SNR10.mat")
print(type(dict_mat))
print(dict_mat.keys())
test = np.matrix.transpose(dict_mat["test"])    #echo+AWGN+jamming
dict_mat = loadmat("C:\\Users\\cccc\\Desktop\\1-DAMRAE\\pure.mat")
testb = np.matrix.transpose(dict_mat["testb"])  #pure echo

S1=test*2-1    #Normalized → -1 to 1
S2=testb*2-1
P1=np.zeros(len(S1))
for i in range(len(S1)):
    P1[i]=np.abs(np.mean(S1[i,:]*S2[i,:]))/np.sqrt(np.abs(np.mean(S1[i,:]*S1[i,:]))*np.abs(np.mean(S2[i,:]*S2[i,:])))
print(np.mean(P1)) #   Waveform similarity between the received signal and the pure echo

test=torch.from_numpy(test).float().view(len(S1),1,2048)
decoder_output= ONEDAMRAE(test)
decoder_output = decoder_output.detach().cpu().numpy()
decoder_output.shape
from scipy.io import savemat
decoder_output.shape
import numpy as np
from scipy import io

S3=decoder_output*2-1
P2=np.zeros(len(S2))
for i in range(len(S2)):
    P2[i]=np.abs(np.mean(S2[i,:]*S3[i,:]))/np.sqrt(np.abs(np.mean(S2[i,:]*S2[i,:]))*np.abs(np.mean(S3[i,:]*S3[i,:])))
print(np.mean(P2)) #   Waveform similarity between the recovered signal and the pure echo

<class 'dict'>
dict_keys(['__header__', '__version__', '__globals__', 'test'])
0.31405597555918485
0.9932307727289404


In [9]:
from scipy.io import loadmat
import numpy as np

dict_mat = loadmat("C:\\Users\\cccc\\Desktop\\1-DAMRAE\\TEST_VV_SJR_neg10_SNR10.mat")
print(type(dict_mat))
print(dict_mat.keys())
test = np.matrix.transpose(dict_mat["test"])    #echo+AWGN+jamming
dict_mat = loadmat("C:\\Users\\cccc\\Desktop\\1-DAMRAE\\pure.mat")
testb = np.matrix.transpose(dict_mat["testb"])  #pure echo

S1=test*2-1    #Normalized → -1 to 1
S2=testb*2-1
P1=np.zeros(len(S1))
for i in range(len(S1)):
    P1[i]=np.abs(np.mean(S1[i,:]*S2[i,:]))/np.sqrt(np.abs(np.mean(S1[i,:]*S1[i,:]))*np.abs(np.mean(S2[i,:]*S2[i,:])))
print(np.mean(P1)) #   Waveform similarity between the received signal and the pure echo

test=torch.from_numpy(test).float().view(len(S1),1,2048)
decoder_output= ONEDAMRAE(test)
decoder_output = decoder_output.detach().cpu().numpy()
decoder_output.shape
from scipy.io import savemat
decoder_output.shape
import numpy as np
from scipy import io

S3=decoder_output*2-1
P2=np.zeros(len(S2))
for i in range(len(S2)):
    P2[i]=np.abs(np.mean(S2[i,:]*S3[i,:]))/np.sqrt(np.abs(np.mean(S2[i,:]*S2[i,:]))*np.abs(np.mean(S3[i,:]*S3[i,:])))
print(np.mean(P2)) #   Waveform similarity between the recovered signal and the pure echo

<class 'dict'>
dict_keys(['__header__', '__version__', '__globals__', 'test'])
0.3764977185838127
0.9938716028584441


In [10]:
from scipy.io import loadmat
import numpy as np

dict_mat = loadmat("C:\\Users\\cccc\\Desktop\\1-DAMRAE\\TEST_DV_SJR_neg10_SNR10.mat")
print(type(dict_mat))
print(dict_mat.keys())
test = np.matrix.transpose(dict_mat["test"])    #echo+AWGN+jamming
dict_mat = loadmat("C:\\Users\\cccc\\Desktop\\1-DAMRAE\\pure.mat")
testb = np.matrix.transpose(dict_mat["testb"])  #pure echo

S1=test*2-1    #Normalized → -1 to 1
S2=testb*2-1
P1=np.zeros(len(S1))
for i in range(len(S1)):
    P1[i]=np.abs(np.mean(S1[i,:]*S2[i,:]))/np.sqrt(np.abs(np.mean(S1[i,:]*S1[i,:]))*np.abs(np.mean(S2[i,:]*S2[i,:])))
print(np.mean(P1)) #   Waveform similarity between the received signal and the pure echo

test=torch.from_numpy(test).float().view(len(S1),1,2048)
decoder_output= ONEDAMRAE(test)
decoder_output = decoder_output.detach().cpu().numpy()
decoder_output.shape
from scipy.io import savemat
decoder_output.shape
import numpy as np
from scipy import io

S3=decoder_output*2-1
P2=np.zeros(len(S2))
for i in range(len(S2)):
    P2[i]=np.abs(np.mean(S2[i,:]*S3[i,:]))/np.sqrt(np.abs(np.mean(S2[i,:]*S2[i,:]))*np.abs(np.mean(S3[i,:]*S3[i,:])))
print(np.mean(P2)) #   Waveform similarity between the recovered signal and the pure echo

<class 'dict'>
dict_keys(['__header__', '__version__', '__globals__', 'test'])
0.31397498464119616
0.9969539508290963


In [11]:
from scipy.io import loadmat
import numpy as np

dict_mat = loadmat("C:\\Users\\cccc\\Desktop\\1-DAMRAE\\TEST_DVV_SJR_neg10_SNR10.mat")
print(type(dict_mat))
print(dict_mat.keys())
test = np.matrix.transpose(dict_mat["test"])    #echo+AWGN+jamming
dict_mat = loadmat("C:\\Users\\cccc\\Desktop\\1-DAMRAE\\pure.mat")
testb = np.matrix.transpose(dict_mat["testb"])  #pure echo

S1=test*2-1    #Normalized → -1 to 1
S2=testb*2-1
P1=np.zeros(len(S1))
for i in range(len(S1)):
    P1[i]=np.abs(np.mean(S1[i,:]*S2[i,:]))/np.sqrt(np.abs(np.mean(S1[i,:]*S1[i,:]))*np.abs(np.mean(S2[i,:]*S2[i,:])))
print(np.mean(P1)) #   Waveform similarity between the received signal and the pure echo

test=torch.from_numpy(test).float().view(len(S1),1,2048)
decoder_output= ONEDAMRAE(test)
decoder_output = decoder_output.detach().cpu().numpy()
decoder_output.shape
from scipy.io import savemat
decoder_output.shape
import numpy as np
from scipy import io

S3=decoder_output*2-1
P2=np.zeros(len(S2))
for i in range(len(S2)):
    P2[i]=np.abs(np.mean(S2[i,:]*S3[i,:]))/np.sqrt(np.abs(np.mean(S2[i,:]*S2[i,:]))*np.abs(np.mean(S3[i,:]*S3[i,:])))
print(np.mean(P2)) #   Waveform similarity between the recovered signal and the pure echo

<class 'dict'>
dict_keys(['__header__', '__version__', '__globals__', 'test'])
0.37640278260776366
0.9966449452061142


In [12]:
from scipy.io import loadmat
import numpy as np

dict_mat = loadmat("C:\\Users\\cccc\\Desktop\\1-DAMRAE\\TEST_OODNPM_SJR_neg8_SNR10.mat")
print(type(dict_mat))
print(dict_mat.keys())
test = np.matrix.transpose(dict_mat["test"])    #echo+AWGN+jamming
dict_mat = loadmat("C:\\Users\\cccc\\Desktop\\1-DAMRAE\\pure.mat")
testb = np.matrix.transpose(dict_mat["testb"])  #pure echo

S1=test*2-1    #Normalized → -1 to 1
S2=testb*2-1
P1=np.zeros(len(S1))
for i in range(len(S1)):
    P1[i]=np.abs(np.mean(S1[i,:]*S2[i,:]))/np.sqrt(np.abs(np.mean(S1[i,:]*S1[i,:]))*np.abs(np.mean(S2[i,:]*S2[i,:])))
print(np.mean(P1)) #   Waveform similarity between the received signal and the pure echo

test=torch.from_numpy(test).float().view(len(S1),1,2048)
decoder_output= ONEDAMRAE(test)
decoder_output = decoder_output.detach().cpu().numpy()
decoder_output.shape
from scipy.io import savemat
decoder_output.shape
import numpy as np
from scipy import io

S3=decoder_output*2-1
P2=np.zeros(len(S2))
for i in range(len(S2)):
    P2[i]=np.abs(np.mean(S2[i,:]*S3[i,:]))/np.sqrt(np.abs(np.mean(S2[i,:]*S2[i,:]))*np.abs(np.mean(S3[i,:]*S3[i,:])))
print(np.mean(P2)) #   Waveform similarity between the recovered signal and the pure echo

<class 'dict'>
dict_keys(['__header__', '__version__', '__globals__', 'test'])
0.38276868741674497
0.9090277899733333


In [13]:
from scipy.io import loadmat
import numpy as np

dict_mat = loadmat("C:\\Users\\cccc\\Desktop\\1-DAMRAE\\TEST_OODCJ_SJR_neg8_SNR10.mat")
print(type(dict_mat))
print(dict_mat.keys())
test = np.matrix.transpose(dict_mat["test"])    #echo+AWGN+jamming
dict_mat = loadmat("C:\\Users\\cccc\\Desktop\\1-DAMRAE\\pure.mat")
testb = np.matrix.transpose(dict_mat["testb"])  #pure echo

S1=test*2-1    #Normalized → -1 to 1
S2=testb*2-1
P1=np.zeros(len(S1))
for i in range(len(S1)):
    P1[i]=np.abs(np.mean(S1[i,:]*S2[i,:]))/np.sqrt(np.abs(np.mean(S1[i,:]*S1[i,:]))*np.abs(np.mean(S2[i,:]*S2[i,:])))
print(np.mean(P1)) #   Waveform similarity between the received signal and the pure echo

test=torch.from_numpy(test).float().view(len(S1),1,2048)
decoder_output= ONEDAMRAE(test)
decoder_output = decoder_output.detach().cpu().numpy()
decoder_output.shape
from scipy.io import savemat
decoder_output.shape
import numpy as np
from scipy import io

S3=decoder_output*2-1
P2=np.zeros(len(S2))
for i in range(len(S2)):
    P2[i]=np.abs(np.mean(S2[i,:]*S3[i,:]))/np.sqrt(np.abs(np.mean(S2[i,:]*S2[i,:]))*np.abs(np.mean(S3[i,:]*S3[i,:])))
print(np.mean(P2)) #   Waveform similarity between the recovered signal and the pure echo

<class 'dict'>
dict_keys(['__header__', '__version__', '__globals__', 'test'])
0.2752818838236201
0.9325394177958714
